In [90]:
import ktrain
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

In [91]:
from ktrain import text

In [92]:
aspirational_df = pd.read_excel('/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/src/data_preprocessing/prepare_sentence_level/aspirational_plus_batch_1_batch_2_merged.xlsx')
aspirational_df

,sentence,label,phrase,batch,original_label,changed?
0,why am i here?,0,"[""ever since i was little i wanted to be a doc...",batch_1,0,False
1,well why does anyone pursue a higher education?,0,"[""ever since i was little i wanted to be a doc...",batch_1,0,False
2,to better one self and be able to succeed late...,1,"[""ever since i was little i wanted to be a doc...",batch_1,1,False
3,ever since i was little i wanted to be a docto...,1,"[""ever since i was little i wanted to be a doc...",batch_1,1,False
4,i always wanted to be able to help people and ...,1,"[""ever since i was little i wanted to be a doc...",batch_1,1,False
...,...,...,...,...,...,...
8715,"anyways, the path im referring to is basically...",0,['one of these goals is being in a career i en...,batch_2,0,False
8716,i know that i want a career in which ill help ...,0,['one of these goals is being in a career i en...,batch_2,0,False
8717,"in any case, i intend to keep moving forward i...",0,['one of these goals is being in a career i en...,batch_2,0,False
8718,i am here to fulfill a covenant with myself an...,1,['i am here to fulfill a covenant with myself ...,batch_2,1,False


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load dataset
file_path = "/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/src/data_preprocessing/prepare_sentence_level/aspirational_plus_batch_1_batch_2_merged.xlsx"
df = pd.read_excel(file_path)

# Ensure 'batch' column is a string
df["batch"] = df["batch"].astype(str)

# Split Batch 1 only
batch1_df = df[df["batch"] == "batch_1"]

# Perform train-test split for Batch 1 (keeping stratification)
train_batch1, test_batch1 = train_test_split(batch1_df, test_size=0.22, random_state=42, stratify=batch1_df["label"])

# Verify counts
print(f"Batch 1 Train: {len(train_batch1)}, Test: {len(test_batch1)}, Total: {len(batch1_df)}")

# Ensure all sentences in Batch 1 train/test are found in Batch 1 + Batch 2
batch1_2_df = df  # Full dataset (Batch 1 + Batch 2)

# Use the same sentences from Batch 1 train in Batch 1 + Batch 2 train
train_batch1_2 = batch1_2_df[batch1_2_df["sentence"].isin(train_batch1["sentence"])]

# Use the same sentences from Batch 1 test i"n Batch 1 + Batch 2 test
test_batch1_2 = batch1_2_df[batch1_2_df["sentence"].isin(test_batch1["sentence"])]

# Verify counts match
print(f"Batch 1 + Batch 2 Train: {len(train_batch1_2)}, Test: {len(test_batch1_2)}, Total: {len(batch1_2_df)}")

# Save final DataFrames
train_batch1.to_excel("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/src/data_preprocessing/prepare_sentence_level/train_batch1.xlsx", index=False)
test_batch1.to_excel("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/src/data_preprocessing/prepare_sentence_level/test_batch1.xlsx", index=False)
train_batch1_2.to_excel("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/src/data_preprocessing/prepare_sentence_level/train_batch1_2.xlsx", index=False)
test_batch1_2.to_excel("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/src/data_preprocessing/prepare_sentence_level/test_batch1_2.xlsx", index=False)

print("✅ Train and test sets successfully created and saved!")

Batch 1 Train: 2727, Test: 770, Total: 3497
Batch 1 + Batch 2 Train: 2893, Test: 909, Total: 8720
✅ Train and test sets successfully created and saved!


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load dataset
file_path = "/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/src/data_preprocessing/prepare_sentence_level/aspirational_plus_batch_1_batch_2_merged.xlsx"
df = pd.read_excel(file_path)

# Ensure 'batch' column is a string
df["batch"] = df["batch"].astype(str)

# Split Batch 1 only
batch1_df = df[df["batch"] == "batch_1"]

# Perform train-test split for Batch 1 (keeping stratification)
train_batch1, test_batch1 = train_test_split(batch1_df, test_size=0.22, random_state=42, stratify=batch1_df["label"])

# Verify counts
print(f"Batch 1 Train: {len(train_batch1)}, Test: {len(test_batch1)}, Total: {len(batch1_df)}")

# Ensure all sentences in Batch 1 train/test are also in Batch 1 + Batch 2
batch1_2_df = df  # Full dataset (Batch 1 + Batch 2)

# --- 🛠️ **Fix for incorrect filtering** ---
# Step 1: Retrieve all Batch 1 sentences from Batch 1+2
train_batch1_2 = batch1_2_df[batch1_2_df["sentence"].isin(train_batch1["sentence"])]
test_batch1_2 = batch1_2_df[batch1_2_df["sentence"].isin(test_batch1["sentence"])]

# Step 2: Add the remaining Batch 2 sentences (not already in train/test)
batch2_df = df[df["batch"] == "batch_2"]
train_batch1_2 = pd.concat([train_batch1_2, batch2_df], ignore_index=True)

# Verify counts match
print(f"Batch 1 + Batch 2 Train: {len(train_batch1_2)}, Test: {len(test_batch1_2)}, Total: {len(batch1_2_df)}")

# # Save final DataFrames
train_batch1.to_excel("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/src/data_preprocessing/prepare_sentence_level/train_batch1.xlsx", index=False)
test_batch1.to_excel("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/src/data_preprocessing/prepare_sentence_level/test_batch1.xlsx", index=False)
train_batch1_2.to_excel("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/src/data_preprocessing/prepare_sentence_level/train_batch1_2.xlsx", index=False)
test_batch1_2.to_excel("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/src/data_preprocessing/prepare_sentence_level/test_batch1_2.xlsx", index=False)

print("✅ Train and test sets successfully corrected and saved!")

Batch 1 Train: 2727, Test: 770, Total: 3497
Batch 1 + Batch 2 Train: 8116, Test: 909, Total: 8720
✅ Train and test sets successfully corrected and saved!


In [7]:
# Count total samples
total_train = len(train_batch1_2)
total_test = len(test_batch1_2)

# Count positive (Class 1) samples
train_positives = train_batch1_2["label"].sum()
test_positives = test_batch1_2["label"].sum()

# Calculate percentages
train_positive_pct = (train_positives / total_train) * 100
test_positive_pct = (test_positives / total_test) * 100

# Print results
print(f"📊 aspirational Dataset Train-Test Split Results:")
print(f"✅ Train: {total_train} samples (Positives: {train_positives}, {train_positive_pct:.2f}%)")
print(f"✅ Test: {total_test} samples (Positives: {test_positives}, {test_positive_pct:.2f}%)")
print(f"🔹 Total Dataset: {len(batch1_2_df)} samples")

📊 aspirational Dataset Train-Test Split Results:
✅ Train: 8116 samples (Positives: 1895, 23.35%)
✅ Test: 909 samples (Positives: 196, 21.56%)
🔹 Total Dataset: 8720 samples


In [110]:
MAXLEN = 150

In [112]:
X = train_batch1_2['sentence']
y = train_batch1_2['label']

In [113]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 18, stratify=y)

In [114]:
model_name = 'distilbert-base-uncased'

In [115]:
distillbert_transformer = text.Transformer(model_name, maxlen=MAXLEN, class_names=[0,1])
training_set = distillbert_transformer.preprocess_train(X_train.tolist(), y_train.tolist())
validation_set = distillbert_transformer.preprocess_test(X_test.tolist(), y_test.tolist())
distillbert_base_model = distillbert_transformer.get_classifier()

preprocessing train...
language: en
train sequence lengths:
	mean : 21
	95percentile : 40
	99percentile : 56


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 21
	95percentile : 40
	99percentile : 53


In [116]:
distillbert_learner = ktrain.get_learner(distillbert_base_model, train_data=training_set, val_data=validation_set, batch_size=6)

In [117]:
distillbert_learner.autofit(2e-5, 5)



begin training using triangular learning rate policy with max lr of 2e-05...
Epoch 1/5
1082/1082 [==============================] - 227s 199ms/step - loss: 0.3437 - accuracy: 0.8524 - val_loss: 0.2927 - val_accuracy: 0.8824
Epoch 2/5
1082/1082 [==============================] - 211s 195ms/step - loss: 0.2199 - accuracy: 0.9145 - val_loss: 0.3029 - val_accuracy: 0.8793
Epoch 3/5
1082/1082 [==============================] - 201s 185ms/step - loss: 0.1554 - accuracy: 0.9432 - val_loss: 0.3330 - val_accuracy: 0.8824
Epoch 4/5
1082/1082 [==============================] - 214s 198ms/step - loss: 0.1060 - accuracy: 0.9650 - val_loss: 0.3728 - val_accuracy: 0.8793
Epoch 5/5
1082/1082 [==============================] - 255s 235ms/step - loss: 0.0726 - accuracy: 0.9769 - val_loss: 0.4266 - val_accuracy: 0.8762


In [118]:
distillbert_learner.validate(class_names=distillbert_transformer.get_classes())

51/51 [==============================] - 17s 250ms/step
              precision    recall  f1-score   support

           0       0.91      0.93      0.92      1245
           1       0.74      0.72      0.73       379

    accuracy                           0.88      1624
   macro avg       0.83      0.82      0.82      1624
weighted avg       0.87      0.88      0.88      1624



array([[1152,   93],
       [ 108,  271]])

In [119]:
distillbert_learner.validate(class_names=distillbert_transformer.get_classes())

51/51 [==============================] - 10s 194ms/step
              precision    recall  f1-score   support

           0       0.91      0.93      0.92      1245
           1       0.74      0.72      0.73       379

    accuracy                           0.88      1624
   macro avg       0.83      0.82      0.82      1624
weighted avg       0.87      0.88      0.88      1624



array([[1152,   93],
       [ 108,  271]])

In [120]:
distillbert_learner.model.summary()

Model: "tf_distil_bert_for_sequence_classification_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_239 (Dropout)       multiple                  0         
                                                                 
Total params: 66955010 (255.41 MB)
Trainable params: 66955010 (255.41 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [121]:
distillbert_predictor = ktrain.get_predictor(distillbert_learner.model, preproc=distillbert_transformer)

In [122]:
distillbert_test_data = test_batch1_2['sentence'].tolist()
distillbert_test_label = test_batch1_2['label'].tolist()

In [123]:
y_pred_distillbert = distillbert_predictor.predict(distillbert_test_data)

In [124]:
y_pred_distillbert = [int(x) for x in y_pred_distillbert]

In [125]:
tn, fp, fn, tp = confusion_matrix(distillbert_test_label, y_pred_distillbert).ravel()
print('True Negative: {}, False Positive: {}, False Negative: {}, True Positive: {}'.format(tn, fp, fn, tp))

True Negative: 655, False Positive: 58, False Negative: 63, True Positive: 133


In [126]:
print('  Classification Report:\n',classification_report(distillbert_test_label,y_pred_distillbert),'\n')

  Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.92      0.92       713
           1       0.70      0.68      0.69       196

    accuracy                           0.87       909
   macro avg       0.80      0.80      0.80       909
weighted avg       0.87      0.87      0.87       909
 



In [25]:
print('  Classification Report:\n',classification_report(distillbert_test_label,y_pred_distillbert),'\n')

  Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.92      0.95       117
           1       0.72      0.92      0.81        25

    accuracy                           0.92       142
   macro avg       0.85      0.92      0.88       142
weighted avg       0.94      0.92      0.93       142
 



In [24]:
#distillbert_predictor.save('./model/distilbert_base_uncased_model') # 256 MB

In [38]:
print("AUC roc score for distillbert model: ", roc_auc_score(distillbert_test_label,y_pred_distillbert))

AUC roc score for distillbert model:  0.8009703180009733
